# Programming Assignment 9: No math-headaches!

<h1 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; top: 60px;right: 30px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" style="width:100px" />
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" style="width:100px" />
</h1>
<h2 style="height: 10px">
</h2>

*[CEGM1000 MUDE](http://mude.citg.tudelft.nl/): Week 2.1. Due: complete this PA prior to class on Friday, Nov 17, 2023.*

## Overview of Assignment

This assignment helps you jumpstart with using SymPy for streamlining any math you'll do during MUDE and for other courses!

This assignment is designed to practise with basic examples. If you don't know how to apply a certain mathematical operation in SymPy, a good start is to take a look at the official documentation.

## Reading

Before completing this assignment, read through and make the exercises in the online textbook [Chapter P-4.3: SymPy](https://mude.citg.tudelft.nl/book/programming/sympy/sympy.html).

## Assignment Criteria

**You will pass this assignment as long as your respository fulfills the following criteria:**  

- You've checked your answers by running the additional cells.
- This notebook exists and runs without errors.

## Task 1: Taylor series

Taylor series require quite a lot of computations, especially for higher order terms. Let's try and implement it to give all the hard work to SymPy!

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 1:</b>   
Given:

$$-x \cdot \cfrac{\cos \left( \pi \cdot x \right) }{3}$$

Find the taylor series around $x = 4$ up to and including the second order using `sympy.series()`.

Tip: take a look at the [documentation](https://docs.sympy.org/latest/modules/series/series.html#sympy.series.series.series).
</p>
</div>

In [1]:
import sympy as sym
from sympy.abc import x
sym.init_printing() 

In [2]:
sym.Symbol('x')
f = -x/3*sym.cos(sym.pi * x) 

# 4 is the value around which the Taylor series is expanded. Choose n=3 for the expansion to be up to x^2.
# .removeO() removes the O notation. 
solution_task_1 = sym.series(f, x, 4, 3).removeO()  
print(solution_task_1)

# To convert the Taylor approximation to an actually evaluable function, I can use the sym.lambdify(x,Taylor_approximation). 
Taylor_approx = sym.lambdify(x, solution_task_1)
print(Taylor_approx(6))

-x/3 + 2*pi**2*(x - 4)**2/3
24.31894506957162


Run the cell below to check your answer:

In [3]:
solution_task_1_solution = 2*sym.pi**2*(x - 4)**2/3 - x/3
check= sym.simplify(solution_task_1.removeO() - solution_task_1_solution) == 0
assert check, "Unfortunately, you're answer is not correct \n \
               Did you use symbolic values for pi? \n \
               Did you evaluate the taylor series around x = 4? \n \
               Did you evaluate the series up to and including the second order? Does `sympy.series` include that order?"
if check:
    print("Well done, you're answer to task 1 is correct!")

Well done, you're answer to task 1 is correct!


## Task 2: Solving differential equations

Solving differential equations can be a pain in the ass, but luckily SymPy can help you with that as well!

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 2a:</b>   

Given:

$$ \cfrac{dy\left( t \right)}{dt} = -100 (y\left( t \right) - \cos(t)) - \sin(t) $$

use `sympy.dsolve` to solve this ODE.

Tips:
- Take a look at the [documentation](https://docs.sympy.org/latest/modules/solvers/ode.html#sympy.solvers.ode.dsolve).
- Define your function $y\left( t \right)$ with `sym.Eq`

Note: the integration constant $C_1$ is defined beforehand as `C1` so that we can solve it later.

</p>
</div>

In [4]:
from sympy import Function, dsolve, Eq, Derivative, sin, cos, symbols, solve
from sympy.abc import t
t = sym.symbols("t")
y = Function('y')

ode = Eq(Derivative(y(t), t), - 100*(y(t) - cos(t)) - sin(t) )
# Alternative way of defining the ODE --> Eq(y.diff(t), -100*y(t- cos(t)) - sym.sin(t))

solution_task_2a = dsolve(ode, y(t))
print(solution_task_2a)

#sym.classify_ode(ode, y(t)) --> useful for choosing the method of solution for the ODE.

Eq(y(t), C1*exp(-100*t) + cos(t))


Run the cell below to check your answer:

In [5]:
C1 = sym.symbols('C1')
solution_task_2a_solution = C1*sym.exp(-100*t) + sym.cos(t)
try:
  solution_task_2a_rhs = solution_task_2a.rhs
except:
  solution_task_2a_rhs = solution_task_2a
check = sym.simplify(solution_task_2a_rhs - solution_task_2a_solution) == 0
assert check, "Unfortunately, you're answer is not correct\n \
               Did you define your function y as a function of t? \n \
               Did you define your goniometric functions from the sympy library?"
if check:
    print("Well done, you're answer to task 2a is correct!")

Well done, you're answer to task 2a is correct!


<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 2b:</b>

Given the inital condition:

$$ y\left(0\right) = 0 $$

Use the solution you found for the task 2a, solve $C_1$ and substitute the numerical value.
Tips:
- Look at the examples in the book to see an example of how to solve unknowns in an equation.
- You've got two values to substitute into your solution for $y \left( t \right)$!
- When substituting your solved value for $C_1$ in your solution for $y$, make sure you the object is a SymPy object.
</p>
</div>

In [6]:
# Method 1: More indirect approach of solving for C1.

# Eq().rhs returns the right-hand side of the expression.
# expression.subs(t,0) substitutes all the t in the expression with 0. 
# solve() solves the expression for C1.
solution_const = solve(solution_task_2a.rhs.subs(t,0), C1)
print(f"C1 equals: {solution_const}. Note that this is an array with 1 element.")


# Substitute C1 in the right-hand side expression with the first element of the solution array.
final_solution = solution_task_2a.rhs.subs(C1, solution_const[0])                  
print(f"Final solution for the function using method 1 is: {final_solution}")


# Method 2: More direct approach of solving for C1.

# This solution uses dsolve and passes the inition conditions directly in the solver.  
solution_task_2b = dsolve(ode, y(t),  ics = {y(0):0})
print(f"Final solution for the function using method 2 is: {solution_task_2b.rhs}")

C1 equals: [-1]. Note that this is an array with 1 element.
Final solution for the function using method 1 is: cos(t) - exp(-100*t)
Final solution for the function using method 2 is: cos(t) - exp(-100*t)


Run the cell below to check your answer:

In [7]:
solution_task_2b_solution = - sym.exp(-100*t) + sym.cos(t)
try:
  solution_task_2b_rhs = solution_task_2b.rhs
except:
  solution_task_2b_rhs = solution_task_2b
check = sym.simplify(solution_task_2b_rhs - solution_task_2b_solution) == 0
assert check,  "Unfortunately, you're answer is not correct \n \
               Did you input both the value of t as the value of y for this given t into your solution for y? \n \
               Did you extract the numerical value from the output of `sym.solve`?"
if check:
    print("Well done, you're answer to task 2b is correct!")

Well done, you're answer to task 2b is correct!


**End of notebook.**
<h2 style="height: 60px">
</h2>
<h3 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; bottom: 60px; right: 50px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
      <img alt="Creative Commons License" style="border-width:; width:88px; height:auto; padding-top:10px" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
    </a>
    <a rel="TU Delft" href="https://www.tudelft.nl/en/ceg">
      <img alt="TU Delft" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png"/>
    </a>
    <a rel="MUDE" href="http://mude.citg.tudelft.nl/">
      <img alt="MUDE" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png"/>
    </a>
    
</h3>
<span style="font-size: 75%">
&copy; Copyright 2023 <a rel="MUDE Team" href="https://studiegids.tudelft.nl/a101_displayCourse.do?course_id=65595">MUDE Teaching Team</a> TU Delft. This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.